## Imports

In [ ]:
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime as dt
from pathlib import Path
from hbt_tools import hydraulics as hd
from hbt_tools import utils_hbt as uh

## Settings

In [ ]:
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")

date_from = dt(2021, 9, 28, 0, 0)
date_to = dt(2022, 10, 1, 0, 0)

id_rfb1 = 459
id_rfb2 = 461
id_Qab = 466
id_Hzu = 465

Dh_rfb1 = 1.727 # m
Dh_rfb2 = 0.926 # m
A_rfb1 = 150 # m2
A_rfb2 = 150 # m2

# Einzeln Beschickung
rfb1_einzeln = [
    ("14.06.2022", "27.06.2022"),
    ("11.10.2022", "08.11.2022")
]

rfb2_einzeln = [
    ("27.06.2022", "04.07.2022"),
    ("08.11.2022", "03.01.2023")
]

## Read data

In [ ]:
h_rfb1 = uh.hbtdb_get_data(id_rfb1, date_from=date_from, date_to=date_to)
h_rfb2 = uh.hbtdb_get_data(id_rfb2, date_from=date_from, date_to=date_to)
h_zu = uh.hbtdb_get_data(id_Hzu, date_from=date_from, date_to=date_to)
Q_ab = uh.hbtdb_get_data(id_Qab, date_from=date_from, date_to=date_to)

## Correct data

In [ ]:
h_rfb1 = (h_rfb1 - Dh_rfb1) * (-1) * 100
h_rfb2 = (h_rfb2 - Dh_rfb2) * (-1) * 100

In [ ]:
mu = 0.6
B = 4 # m, Breite, Schalungsplan
h0 = 1.05 # m, anspringswert gemäss Christian

h_zu = h_zu / 100 - h0 # Höhe über Überlaufkante
Q_zu = h_zu.copy()

Q_zu.iloc[:,0] = Q_zu.iloc[:,0].clip(lower=0).apply(lambda x:
    hd.poleni_q(mu, B, x)) * 1000

## Plot data

### RFB 1

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=Q_zu.index, y=Q_zu.iloc[:,0], name='Q Zulauf'),
    secondary_y=True)
fig.add_trace(go.Scatter(x=Q_ab.index, y=Q_ab.iloc[:,0], name='Q Ablauf'),
    secondary_y=True)
fig.add_trace(go.Scatter(x=h_rfb1.index, y=h_rfb1.iloc[:,0], name='h RFB 1'),
    secondary_y=False)

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Niveau [cm]', secondary_y=False)
fig.update_yaxes(title='Durchfluss [l/s]', secondary_y=True)

fig.update_layout(title='RFB 1', font=dict(size=25))
fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

save_path = save_dir / 'Filterleistung_RFB-1.html'
fig.write_html(save_path)

### RFB 2

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=Q_zu.index, y=Q_zu.iloc[:,0], name='Q Zulauf'),
    secondary_y=True)
fig.add_trace(go.Scatter(x=Q_ab.index, y=Q_ab.iloc[:,0], name='Q Ablauf'),
    secondary_y=True)
fig.add_trace(go.Scatter(x=h_rfb2.index, y=h_rfb2.iloc[:,0], name='h RFB 2'),
    secondary_y=False)

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Niveau [cm]', secondary_y=False)
fig.update_yaxes(title='Durchfluss [l/s]', secondary_y=True)

fig.update_layout(title='RFB 2', font=dict(size=25))
fig.update_xaxes(title=dict(font=dict(size=25)))
fig.update_yaxes(title=dict(font=dict(size=25)))

save_path = save_dir / 'Filterleistung_RFB-2.html'
fig.write_html(save_path)